In [ ]:
"""
Whois

Data required: Authoritative nameservers, Trigrams in domain name,
IP addresses of nameservers, Registrar, ASes of nameserver IP addresses

Raw data: Authoritative nameservers, Registrar?
Enrich: Trigrams in domain name, IP addresses of nameservers, ASes of nameserver IP addresses, /16 mask
"""

In [ ]:
"""
SNA

Data required: Subdomain
Enrich: IP, Domain, ASN, Network

IP - DNS query type A
Nameserver - DNS query type NS

IP to Domain – 1 divided by the amount of domains hosted in the given 

Domain to IP – 1 divided by the amount of IP who host the given Domain 

IP to IP – 1 divided by the amount of the IPs common attributes, when common (ASN, /16 Network) 

Domain to Domain – 1 divided by the amount of all domains with the same parent domain
"""

In [1]:
import pandas as pd
import networkx as nx

In [ ]:
ip_str = '123.23.143.98'
find_nth(ip_str, '.', 2)

In [72]:
l = ['a', 'b', 'c']
test_df = pd.DataFrame(l, columns=['t1'])

In [73]:
test_df

,t1
0,a
1,b
2,c


In [194]:
alexa_df = pd.read_csv('resources/top-1m.csv', names=['rank', 'domain'])

In [195]:
df = alexa_df[0:1000]

In [163]:
df = pd.read_csv('top_benign.csv')

In [196]:
%%time
# domain features
df.loc[:, 'domain_name'] = df['domain'].apply(get_domain_name)
df.loc[:, 'base_domain'] = df['domain'].apply(get_base_domain)
df.loc[:, 'trigrams'] = df['domain_name'].apply(extract_trigrams)
df.loc[:, 'domain_ip'] = df['domain'].apply(get_domain_ip)

df.loc[:, 'as_subnet'] = df['domain_ip'].apply(get_ASN_and_subnet_by_IP)
df.loc[:, 'as_number'] = df['as_subnet'].apply(lambda x: x[0])
df.loc[:, 'as_subnet'] = df['as_subnet'].apply(lambda x: x[1])
df.loc[:, 'as_name'] = df['as_number'].apply(get_as_name)

# name server features
df.loc[:, 'name_server'] = df['base_domain'].apply(get_domain_name_server)
df.loc[:, 'ns_base_domain'] = df['name_server'].apply(get_base_domain)
df.loc[:, 'ns_ip'] = df['name_server'].apply(get_domain_ip)
df.loc[:, 'ns_as_subnet'] = df['ns_ip'].apply(get_ASN_and_subnet_by_IP)
df.loc[:, 'ns_as_number'] = df['ns_as_subnet'].apply(lambda x: x[0])
df.loc[:, 'ns_as_subnet'] = df['ns_as_subnet'].apply(lambda x: x[1])
df.loc[:, 'ns_as_name'] = df['ns_as_number'].apply(get_as_name)
df.loc[:, 'label'] = 0

df.to_csv('top_benign.csv', index=False)
# df['network_16'] = df['domain_ip'].apply(get_network_address)

c:\users\giliv\appdata\local\programs\python\python37-32\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
c:\users\giliv\appdata\local\programs\python\python37-32\lib\site-packages\pandas\core\indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


twimg.com - NX domain
cbbp1.com - NX domain
babytree.com - NX domain
cdninstagram.com - NX domain
bp.blogspot.com - NX domain
cloudfront.net - NX domain
17ok.com - NX domain
wixsite.com - NX domain
nhk.or.jp - NX domain
eshkol.io - NX domain
japanpost.jp - NX domain
ytimg.com - NX domain
notifications.website - NX domain
romedy.pro - NX domain
dy2018.com - NX domain
searchred.club - NX domain
doorblog.jp - NX domain
akamaized.net - NX domain
tsinghua.edu.cn - NX domain
intentmedia.net - NX domain
soso.com - NX domain
douyu.com - NX domain
rednet.cn - NX domain
momoshop.com.tw - NX domain
dytt8.net - NX domain
www.gov.uk - NX domain
searchred.club - NX domain
52pojie.cn - NX domain
error get base domain of None
error get base domain of None
error get base domain of None
error get base domain of None
error get base domain of None
error get base domain of None
error get base domain of None
error get base domain of None
None - NX domain
None - NX domain
None - NX domain
None - NX domain
No

In [93]:
df.to_csv('top_benign2.csv', index=False)

In [62]:
phishing_df = pd.read_csv('resources/phishing.csv', names=['domain', 'count'], header=0)

In [130]:
G = nx.Graph()

def append_row_to_graph(row):
    G.add_node(row['domain'], start=row['label'], current=row['label'])
    G.add_node(row['domain_ip'], start=row['label'], current=row['label'])
    G.add_node(row['as_subnet'], start=0.5, current=0.5)
    G.add_node(row['as_number'], start=0.5, current=0.5)
    G.add_node(row['as_name'], start=0.5, current=0.5)
    G.add_node(row['ns_base_domain'], start=0.5, current=0.5)
    G.add_node(row['ns_as_subnet'], start=0.5, current=0.5)
    G.add_node(row['ns_as_number'], start=0.5, current=0.5)
    G.add_node(row['ns_as_name'], start=0.5, current=0.5)
    
    G.add_edge(row['domain'], row['domain_ip'])
    G.add_edge(row['domain_ip'], row['as_subnet'])
    G.add_edge(row['as_subnet'], row['as_number'])
    G.add_edge(row['as_number'], row['as_name'])
    
    G.add_edge(row['domain'], row['ns_base_domain'])
    G.add_edge(row['ns_base_domain'], row['ns_as_subnet'])
    G.add_edge(row['ns_as_subnet'], row['ns_as_number'])
    G.add_edge(row['ns_as_number'], row['ns_as_name'])
    
    if None in G:
        G.remove_node(None)
    

In [132]:
len(G)

2130

In [131]:
%%time
_ = df.apply(append_row_to_graph, axis=1)

Wall time: 127 ms


In [123]:
def update_node(G, node):
    if G.degree(node) > 0:
        neighbors_current_score = 0
        for neighbor in G.neighbors(node):
            neighbors_current_score += G.nodes[neighbor]['current']
        neighbors_current_avg = neighbors_current_score / G.degree(node)
        G.nodes[node]['current'] = 0.5 * G.nodes[node]['current'] + 0.5 * neighbors_current_avg

def graph_iteration(G):
    H = G.copy()
    for node, data in H.nodes(data=True):
        update_node(G, node)
#         if H.degree(node) > 0:
#             neighbors_current_score = 0
#             for neighbor in H.neighbors(node):
#                 neighbors_current_score += H.nodes[neighbor]['current']
#             neighbors_current_avg = neighbors_current_score / H.degree(node)
#         G.nodes[node]['current'] = 0.5 * G.nodes[node]['current'] + 0.5 * neighbors_current_avg

In [135]:
%%time
H = G.copy()
for i in range(10):
    graph_iteration(H)
    print (i, (H.nodes['google.com']['current']))

0 0.28205128205128205
1 0.20225648024414244
2 0.1735570704840475
3 0.16413197879804098
4 0.16206685783177577
5 0.16277152205079815
6 0.16442539137445852
7 0.16629179972923674
8 0.16807524744205454
9 0.16966676129998443
Wall time: 305 ms


In [162]:
df.head(1)

,rank,domain,domain_name,base_domain,trigrams,domain_ip,as_subnet,as_number,as_name,name_server,...,domain_as_number_tony.ns.cloudflare.com,domain_as_number_udns2.ultradns.net,domain_as_number_use1.akam.net,domain_as_number_v2.dnsip.net,domain_as_number_vern.ns.cloudflare.com,domain_as_number_vip3.alidns.com,domain_as_number_vita.ns.cloudflare.com,domain_as_number_wally.ns.cloudflare.com,domain_as_number_will.ns.cloudflare.com,domain_as_number_zara.ns.cloudflare.com
0,1,google.com,google,google.com,"{goo, gle, ogl, oog}",216.58.206.14,216.58.206.0/23,15169,"GOOGLE - Google LLC, US",ns3.google.com,...,0,0,0,0,0,0,0,0,0,0


In [178]:
### ML DF
categorical_feature_list = ['as_number', 'as_subnet', 'as_name', 'ns_base_domain', 'ns_as_subnet', 'ns_as_number', 'ns_as_name']


X = df[['domain']]
for feature in categorical_feature_list:
    X = X.join(pd.get_dummies(df[feature], prefix=feature))
    print (len(X.columns))

183
549
728
957
1169
1297
1421


In [190]:
pd.DataFrame(df.trigrams.values.tolist(), index=df.index)

,0
0,"{'goo', 'gle', 'ogl', 'oog'}"
1,"{'utu', 'you', 'tub', 'ube', 'out'}"
2,"{'ebo', 'ceb', 'ook', 'boo', 'fac', 'ace'}"
3,"{'idu', 'aid', 'bai'}"
4,"{'ipe', 'iki', 'wik', 'dia', 'edi', 'ped', 'kip'}"
5,set()
6,"{'bao', 'oba', 'aob', 'tao'}"
7,"{'azo', 'zon', 'maz', 'ama'}"
8,"{'mal', 'tma', 'all'}"
9,"{'yah', 'aho', 'hoo'}"


In [193]:
df.trigrams.values[0]

"{'goo', 'gle', 'ogl', 'oog'}"

In [187]:
df['trigrams'].apply(pd.Series)

,0
0,"{'goo', 'gle', 'ogl', 'oog'}"
1,"{'utu', 'you', 'tub', 'ube', 'out'}"
2,"{'ebo', 'ceb', 'ook', 'boo', 'fac', 'ace'}"
3,"{'idu', 'aid', 'bai'}"
4,"{'ipe', 'iki', 'wik', 'dia', 'edi', 'ped', 'kip'}"
5,set()
6,"{'bao', 'oba', 'aob', 'tao'}"
7,"{'azo', 'zon', 'maz', 'ama'}"
8,"{'mal', 'tma', 'all'}"
9,"{'yah', 'aho', 'hoo'}"


In [185]:
pd.get_dummies(df['trigrams'])

,set(),"{'133', '37x', '337'}",{'163'},{'178'},"{'1jm', '91j'}",{'360'},"{'365', 'bet', 't36', 'et3'}","{'470', '700', 'n47', 'r9n', '9n4', '04g', '004', 'yr9'}","{'587', '87t', 'xj1', 'txj', '6b5', 'l6b', 'b58', '7tx'}","{'5v1', 'qh5', '1qh', '71q', 'jf7', 'h5v', 'v14', 'f71'}",...,"{'yah', 'aho', 'hoo'}","{'yel', 'elp'}","{'yma', 'ail', 'ily', 'mai', 'dai', 'lym'}","{'you', 'uku', 'ouk'}","{'you', 'um7', 'oum'}",{'yts'},"{'yv9', '0by', 'mgb', 'bn0', 'byv', 'v9m', 'gbn', '9mg'}","{'zfe', 'buz', 'zzf', 'uzz', 'fee', 'eed'}","{'zma', 'eti', 'izm', 'ket', 'hiz', 'ark', 'whi', 'rke', 'ing', 'the', 'ewh', 'tin', 'hew', 'mar'}",{'zol'}
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [ ]:
H.nodes['igcdn.com']['start'] 

In [ ]:
df[df['domain'] == 'cdninstagram.com']

In [10]:
import dns.resolver
import tldextract
import pyasn
from netaddr import IPNetwork
from pyasn_util_asnames import download_asnames, _html_to_dict

asndb = pyasn.pyasn('resources/asn20190719.db')
data = download_asnames()
asn_dict = _html_to_dict(data)

def extract_trigrams(domain_name):
    trigrams = set()
    for i in range(0, len(domain_name) - 2):
        trigrams.add(domain_name[i:i+3])
    return trigrams


def get_domain_ip(domain):
    ip = None
    try:
        res = dns.resolver.query(domain, 'A')
        ip = res[0].address
    except:
        print (f'{domain} - NX domain')
    return ip


def get_domain_name_server(domain):
    ns = None
    try:
        res = dns.resolver.query(domain, 'NS')
        ns = res[0].to_text()[:-1]
    except:
        print (f'{domain} - NX domain')
    return ns


def get_base_domain(domain):
    base_domain = None
    try:
        exr = tldextract.extract(domain)
        base_domain = f'{exr.domain}.{exr.suffix}'
    except:
        print (f'error get base domain of {domain}')
    return base_domain


def get_domain_name(domain):
    domain_name = None
    try:
        exr = tldextract.extract(domain)
        domain_name = exr.domain
    except:
        print (f'error get domain name of {domain}')
    return domain_name


def get_ASN_and_subnet_by_IP(ip):
    as_number = None
    subnet = None
    if ip:
        try:
            asn, subnet = asndb.lookup(ip)
            as_number = str(asn)
        except:
            print(f'cannot resolve ASN for ip: {ip}')
    return as_number, subnet


def get_network_address(ip, mask=16):
    network = None
    try:
        network = str(IPNetwork(f'{ip}/{mask}').cidr)
    except:
        print(f'cannot resolve network for ip: {ip}')
    return network


def get_as_name(as_number):
    as_name = None
    if as_number and as_number in asn_dict:
        as_name = asn_dict[as_number]
    return as_name


In [ ]:
ipaddress.ip_address('192.168.0.1') in ipaddress.ip_network('192.168.0.0/24')

In [ ]:
ipaddress.ip_network('123.232.0.0/16')